In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Define the URLs for the links
url1 = "/lab/"
url2 = "/voila/render/ui/conn.ipynb"

# Define the button labels
button_label1 = "Go to Home Page"
button_label2 = "Go to Connection"

# Define HTML links for buttons
button_html1 = f"<a href='{url1}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px; margin-right: 10px;'><i class='fa fa-home'></i></a>"
button_html2 = f"<a href='{url2}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px;'>{button_label2}</a>"


# Create HTML widgets
html_button1 = widgets.HTML(value=button_html1)
html_button2 = widgets.HTML(value=button_html2)

# Display buttons
display(widgets.HBox([html_button1, html_button2]))


_______

In [ ]:
import os
import logging

log_folder = 'logs'
if not os.path.exists(log_folder):
    os.makedirs(log_folder)
logging.basicConfig(filename='logs/jupyter_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
logging.info('[Query Profile Main Page] Importing Libraries')
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import verticapy as vp
from IPython.display import display,clear_output, IFrame, HTML
import pickle
import re
import webbrowser
from ipyfilechooser import FileChooser
import verticapy as vp
from verticapy.performance.vertica import QueryProfilerInterface
from verticapy.connection.errors import ConnectionError
vp.set_option("print_info", False)
logging.info('[Query Profile Main Page] Successfully Imported Libraries')

In [ ]:
def reset_values(query_val = None, key_val = None, target_schema_val = None, transaction_val = None, statement_val = None, file_val = None, key_val_2 = None, target_schema_val_2 = None, transaction_statement_list_val = None):    
    if query_val:
        %store query_val
    else:
        %store -d query_val
    if key_val:
        %store key_val
    else: 
        %store -d key_val
    if target_schema_val:
        %store target_schema_val
    else:
        %store -d target_schema_val
    if transaction_val:
        %store transaction_val
    else:
        %store -d transaction_val
    if statement_val:
        %store statement_val
    else:
        %store -d statement_val
    if file_val:
        %store file_val
    else:
        %store -d file_val
    if key_val_2:
        %store key_val_2
    else: 
        %store -d key_val_2
    if target_schema_val_2:
        %store target_schema_val_2
    else:
        %store -d target_schema_val_2
    if transaction_statement_list_val:
        %store transaction_statement_list_val
    else:
        %store -d transaction_statement_list_val
    
    notebook_url = '/voila/render/ui/qprof-ui.ipynb'
    display(HTML(f"<script>window.open('{notebook_url}');window.close();</script>"))

In [ ]:
%%html
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        .header-container {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 10px;
        }
        .header-container h1,
        .header-container a {
            margin: 0;
            margin-left: 42px;
        }
    </style>
    <title>Connect to Vertica</title>
</head>
<body>
    <div class="header-container">
        <h1>Profile your queries</h1>
        <a href="https://www.vertica.com/python/documentation/1.0.x/html/api/verticapy.performance.vertica.qprof.QueryProfiler.html">Click here for more information on QueryProfiler</a>
    </div>
    <div style="margin-left: 42px; padding:10px; font-size: 1rem">
        Get query profile information using an existing saved QueryProfile object or create one from scratch.
    </div>
</body>
</html>

In [ ]:
%%html
<style>
    .custom_class_conn_info_header {
        margin: 0;
        padding: 0px 0px 0px 10px;
        margin-left: 42px;
    }
</style>
<div class = "custom_class_conn_info_header">
<h3> Your connection details: </h3>
</div>

In [ ]:
logging.info('[Query Profile Main Page] Trying to print database connection details')
conn_output = widgets.Output(layout={"margin" :"0px 50px 0px 50px"})
with conn_output:
    try:
        vdf_test_sandbox = vp.vDataFrame("select distinct sandbox from nodes where node_state='UP' and sandbox is not NULL;")
        if vdf_test_sandbox[0] is None or vdf_test_sandbox[0][0] =='':
            print("\033[1mDatabase\033[0m:", vp.current_connection().options['database'], "    \033[1mUsername\033[0m:", vp.current_connection().options['user'])
        else:
            print("\033[1mDatabase\033[0m:", vp.current_connection().options['database'],
                  "    \033[1mUsername\033[0m:", vp.current_connection().options['user'],
                  "    \033[1mSandbox\033[0m:", vdf_test_sandbox[0][0])
    except ConnectionError:
        print("\033[91m\033[1m WARNING: No database connection found. Please connect with a database to use this tool. \033[0m\033[0m")
display(conn_output)
logging.info('[Query Profile Main Page] Done with printing database connection details ')

____

In [ ]:
# Function to update the options of the key dropdown based on the selection of the schema dropdown
def update_key_dropdown_options(change):
    # Get the selected schema from the first dropdown
    selected_schema = change.new
    # Manipulate table names
    length_to_remove = len("qprof_dc_explain_plans_") + 1
    # Query for the table names based on the selected schema
    query = f"SELECT SUBSTR(table_name, {length_to_remove}) AS 'table_name' FROM (SELECT DISTINCT table_name FROM tables WHERE table_name LIKE '%_dc_explain_plans_%' AND table_schema = '{selected_schema}') as foo order by 1;"
    vdf = vp.vDataFrame(query)
    # Update options of the second dropdown
    key_dropdown.options = [i[0] for i in vdf.to_list()]

# Function to observe the value of the key and target_schema widget and disable both dropdowns if there is any user value
def observe_value_change(change):
    # If the key widget or target_schema widget has any user value, disable both dropdowns
    if key.value or target_schema.value:
        schema_dropdown.disabled = True
        key_dropdown.disabled = True
    else:
        schema_dropdown.disabled = False
        key_dropdown.disabled = False


# Query for the options of the first dropdown
query = "SELECT DISTINCT table_schema FROM tables WHERE table_name LIKE '%_dc_explain_plans_%' ORDER BY 1;"
vdf = vp.vDataFrame(query)
schema_dropdown_options = [i[0] for i in vdf.to_list()]

# Create the first dropdown
schema_dropdown = widgets.Dropdown(
    options=schema_dropdown_options,
    value = None,
    description='Schema List',
    disabled=False,
    layout={'width': "250px"}
)
schema_dropdown.style.description_width = '100px'

# Create the second dropdown with initial options
key_dropdown = widgets.Dropdown(
    options=[],  # Initial options, will be updated dynamically
    description='Key List',
    disabled=False,
    layout={'width': "250px"}
)
key_dropdown.style.description_width = '100px'

# Observe changes in the first dropdown and trigger the update function
schema_dropdown.observe(update_key_dropdown_options, names='value')

In [ ]:
def schema_exists(schema_name):
    query = f"SELECT schema_name FROM v_catalog.schemata WHERE schema_name = '{schema_name}';"
    vdf = vp.vDataFrame(query)
    return vdf.shape()[0] > 0

In [ ]:
button_2 = widgets.Button(description="Get Query Plan")
output_2 = widgets.Output()

key_val = None
target_schema_val = None

def on_button_clicked_2(b):
    global key_val, target_schema_val
    output_2.clear_output(wait=True)
    with output_2:
        key_val = key.value if key.value else key_dropdown.value
        target_schema_val = target_schema.value if target_schema.value else schema_dropdown.value
        if not schema_exists(target_schema_val):
            print("\033[91m\033[1m Error! The schema does not exist. \033[0m\033[0m")
        else:
            logging.info(f"[Query Profile Main Page] For load from schema and key option, saved key value as :{key_val}, target schema as {target_schema_val}. ")
            reset_values(key_val = key_val, target_schema_val = target_schema_val)

button_2.on_click(on_button_clicked_2)


button_3 = widgets.Button(description="Get Query Plan")
file_val = None
output_3 = widgets.Output()

def on_button_clicked_3(b):
    global key_val, target_schema_val, file_val
    output_3.clear_output(wait=True)
    with output_3:
        key_val = key.value if key.value else key_dropdown.value
        if key_val == False:
            key_val = None
        target_schema_val = target_schema.value if target_schema.value else schema_dropdown.value
        file_val = uploader.value
        logging.info(f"[Query Profile Main Page] For load from file option, saved key value as :{key_val}, target schema as {target_schema_val}, file name as {file_val}.")
        if target_schema_val == "" or file_val == "" or target_schema_val == None:
            print("\033[91m\033[1m Error! You must provide a value for Schema and File. \033[0m\033[0m")
            return
        elif check_if_schema_key_exist(target_schema_val, key_val):
            print("\033[91m\033[1m Error! The combination of schema and key already exist. Please try again with another key that is unique. \033[0m\033[0m")
            # print("Error! The combination of schema and key already exist. Please try again with another key that is unique.")
            return
        else:
            reset_values(key_val = key_val, target_schema_val = target_schema_val, file_val = file_val)

button_3.on_click(on_button_clicked_3)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def create_tooltip(text):
    # Create an info icon using HTML
    info_icon = widgets.HTML(
        value=f'<span style="font-size: 25px; color: #007bff; cursor: pointer;" title="{text}">&#9432;</span>'
    )
    return info_icon

In [ ]:
html_arrow = """
<style>
.container {
    display: flex;
    align-items: center; /* Center vertically */
    padding-left: 5px; /* Add padding to the left */
}

.arrow {
    font-weight: bold; /* Make the arrow bold */
    font-size: 2em; /* Make the arrow slightly bigger */
}
</style>

<div class="container">
    Or select from a list
    <span class="arrow">&rarr;</span> 
</div>
"""

In [ ]:
key = widgets.Text(description = "Key", placeholder = "Enter Key", layout=widgets.Layout(width='300px'))
key.style.description_width = '100px'
target_schema = widgets.Text(description = "Target Schema", placeholder = "Enter Schema",layout=widgets.Layout(width='300px'))
target_schema.style.description_width = '100px'
uploader = FileChooser('')
uploader_combo = widgets.HBox([uploader, create_tooltip("Select the tar file that you obtained after exporting the query profile object.")])
from ipyfilechooser import FileChooser

# Observe changes in the key and target_schema widget to disable both dropdowns if there is any user value
key.observe(observe_value_change, names='value')
target_schema.observe(observe_value_change, names='value')

tab = widgets.Tab()
target_schema_combo = widgets.HBox([target_schema,widgets.HTML(html_arrow), schema_dropdown, create_tooltip("Enter the target schema that you used to save the QueryProfiler object. This will not work if you have not saved your query profile already.")])
key_combo = widgets.HBox([key, widgets.HTML(html_arrow), key_dropdown, create_tooltip("Enter the unique key that you used to save the QueryProfiler object. This will not work if you have not saved your query profile already.")])
child_1 = widgets.VBox([target_schema_combo,key_combo,button_2, output_2])
child_2 = widgets.VBox([target_schema_combo,key_combo,uploader_combo,button_3, output_3])
tab.children = [child_1, child_2]
tab.titles = ["From database", "From a file"]
logging.info('[Query Profile Main Page] Trying to display the load tab')
logging.info('[Query Profile Main Page] Successfully loaded')
left_heading = widgets.HTML("""<h1 style="text-align: left;">Load an existing QueryProfiler</h1>""")
left_section = widgets.VBox([left_heading,tab], layout= {"margin" :"0px 0px 0px 50px"})

In [ ]:
search_box_test = widgets.HTML("""
You can query the 'QUERY_REQUESTS' table to search for the required query to get the transaction ID and statement ID needed to profile the query. You can use either:
<ul>
<li>Reqeuest label</li> 
<li>Query part</li>
</ul>
For more information, please refer to the <a href="https://docs.vertica.com/24.2.x/en/sql-reference/system-tables/v-monitor-schema/query-requests/" style="color: blue; text-decoration: underline;">query_requests documentation</a>.
""")

In [ ]:
request_label_input = widgets.Text(description = "Request label", placeholder = "Enter part of label")
request_label_check = widgets.Checkbox(value=False,description='Exact')
query_search_input = widgets.Text(description = "Query text", placeholder = "Enter part of query")
query_search_check = widgets.Checkbox(value=False,description='Exact')
button_01 = widgets.Button(description="Query Request Table", layout=widgets.Layout(width='auto'))
output_01 = widgets.Output()
request_label_widget = widgets.HBox([request_label_input, 
                                     create_tooltip("Labels are inserted inside the query in order to track them. If you have labeled your queries, you can conveniently serach them from their labels."), 
                                     request_label_check])
query_search_widget = widgets.HBox([query_search_input, 
                                    create_tooltip("Enter the exact Query to search or part of it."),
                                    query_search_check])
search_box_vbox = widgets.VBox([search_box_test, request_label_widget, query_search_widget, button_01, output_01])

query_session_control_input = widgets.Text(description="SQL:", placeholder="Enter Session Control SQL",
    layout={'width': "650px"})
query_session_control_button = widgets.Button(description="Run SQL")
query_session_control_output = widgets.Output()

def on_query_session_control_button_clicked(b):
    query_session_control_button.disabled = True
    query_session_control_output.clear_output(wait=True)
    with query_session_control_output:
        sql_query = query_session_control_input.value
        if sql_query:
            logging.info(f'[Query Profile Main Page] Running the following query for session parameter changes: {sql_query}')
            vp._utils._sql._sys._executeSQL(sql_query)
            print("Successfully ran the query for session parameter changes.")
            logging.info('[Query Profile Main Page] Successfully fran query for session parameter chagnes')
        else:
            print("\033[91m\033[1m Error! Please enter a SQL statement. \033[0m\033[0m")
    query_session_control_button.disabled = False

query_session_control_button.on_click(on_query_session_control_button_clicked)

query_session_control_box = widgets.VBox([
    query_session_control_input,
    query_session_control_button,
    query_session_control_output
])

# Update the accordion with the new child
search_box = widgets.Accordion(children=[search_box_vbox, query_session_control_box], titles=('Search for a query', 'Session Parameter Control'))


def on_button_clicked_01(b):
    output_01.clear_output(wait=True)
    with output_01:
        if not request_label_check.value:
            request_label_wild_card = '%'
        else:
            request_label_wild_card = ''
        if not query_search_check.value:
            query_search_wild_card = '%'
        else:
            query_search_wild_card = ''
        logging.info(f'[Query Profile Main Page] Trying to fetch the queries from query_request table using the label: {request_label_input.value}, and query text: {query_search_input.value}')
        sql_query = f"SELECT * from query_requests WHERE request_type = 'QUERY' AND request_label LIKE '{request_label_wild_card}{request_label_input.value}{request_label_wild_card}' AND request LIKE '{query_search_wild_card}{query_search_input.value}{query_search_wild_card}' ORDER BY request_duration_ms DESC LIMIT 50;"
        result = vp.vDataFrame(sql_query)
        logging.info('[Query Profile Main Page] Successfully fetched the queries from query_request table')
        result.idisplay()
        
logging.info('Trying to display Query Request Table button [Query Profile Main Page]')
button_01.on_click(on_button_clicked_01)
logging.info('Loaded Query Request Table button [Query Profile Main Page]')

In [ ]:
transaction = widgets.Text(description = "Transaction ID", placeholder = "Enter a Transaction ID", layout=widgets.Layout(width='300px'))
transaction.style.description_width = '100px'
transaction_combo = widgets.HBox([transaction, create_tooltip("Enter the transaction ID of the query that you want to profile. You can search for the transaction ID from the search box above.")])
statement = widgets.Text(description = "Statement ID", placeholder = "Enter a Statement ID", layout=widgets.Layout(width='300px'))
statement.style.description_width = '100px'
statement_combo = widgets.HBox([statement, create_tooltip("Enter the statement ID of the query that you want to profile. You can search for the statement ID from the search box above.")])


In [ ]:
key_2 = widgets.Text(description = "Key (Optional)", placeholder = "Enter a unique Key", layout=widgets.Layout(width='300px'))
key_2.style.description_width = '100px'
key_2_combo = widgets.HBox([key_2, create_tooltip("Enter a unique key that will be used to save your profile tables. If your key is already in use then a random key will be generated to store the data. Note that you will need the combination of schema and the key to load your saved data. If you do not provide this, you may not be able to export your profile tables to a tar file.")])
target_schema_2 = widgets.Text(description = "Target Schema (Optional)", placeholder = "Enter Schema",layout=widgets.Layout(width='300px'))
target_schema_2.style.description_width = '150px'
target_schema_2_combo = widgets.HBox([target_schema_2, create_tooltip("Enter the schema in which you would like to store the profile data. Note that you will need the combination of schema and the key to load your saved data. If you do not provide this, you may not be able to export your profile tables to a tar file.")])

In [ ]:
button_1 = widgets.Button(description="Get Query Plan")
output_1 = widgets.Output()
transaction_val = None
statement_val = None
target_schema_val_2 = None
key_val_2 = None

def on_button_clicked_1(b):
    output_1.clear_output(wait=True)
    global target_schema_val_2, key_val_2, statement_val, transaction_val
    with output_1:
        target_schema_val_2 = target_schema_2.value
        if not target_schema_val_2 == "" and not schema_exists(target_schema_val_2):
            print("\033[91m\033[1m Error! The schema does not exist. \033[0m\033[0m")
        else:
            key_val_2 = key_2.value
            transaction_val = transaction.value
            statement_val = statement.value
            logging.info(f"[Query Profile Main Page] For create option, saved key value as :{key_val_2}, target schema as {target_schema_val_2}, transaction id as {transaction_val}, statement id as {statement_val}.")
            reset_values(statement_val = statement_val, transaction_val = transaction_val, target_schema_val_2 = target_schema_val_2, key_val_2 = key_val_2)

button_1.on_click(on_button_clicked_1)
create_0 = widgets.VBox([target_schema_2_combo, key_2_combo,transaction_combo,statement_combo, button_1, output_1])

In [ ]:
query = widgets.Textarea(
    placeholder='Enter a SQL Query',
    description='SQL Query:',
    disabled=False,
    layout=widgets.Layout(width='700px', height = '300px')
)

In [ ]:
button = widgets.Button(description="Get Query Plan")
output_0 = widgets.Output()
query_val = None

# Split the queries
def query_list(sql_queries: str):
    # Regular expression to match SQL queries
    pattern = r'((?:[^;"\']|"(?:\\.|[^"])*"|\'(?:\\.|[^\'])*\')+);?\s*'
    queries = [query.strip() for query in re.findall(pattern, sql_queries) if query.strip()]
    return queries


# Check if there is at least one space inside the query - to avoid the ";;" typo mistake.
def query_check(queries: list):
    for query in queries:
        if ' ' not in query:
            return 1
    return 0
            

def on_button_clicked_0(b):
    output_0.clear_output(wait=True)
    global query_val, target_schema_val_2, key_val_2
    with output_0:
        target_schema_val_2 = target_schema_2.value
        query_val = query_list(query.value)
        if not target_schema_val_2 == "" and not schema_exists(target_schema_val_2):
            print("\033[91m\033[1m Error! The schema does not exist. \033[0m\033[0m")
        elif query_check(query_val):
            print("\033[91m\033[1m Error! Re-check your query. \033[0m\033[0m")
        else:
            key_val_2 = key_2.value
            logging.info(f"""[Query Profile Main Page] For create using query option, saved key value as :{key_val_2}, target schema as {target_schema_val_2}, query as "{query_val}".""")
            reset_values(query_val = query_val, target_schema_val_2 = target_schema_val_2, key_val_2 = key_val_2)

button.on_click(on_button_clicked_0)
create_1 = widgets.VBox([target_schema_2_combo, key_2_combo, query, button, output_0])


In [ ]:
transaction_statement_list = widgets.Text(description = "List of transaction IDs and statement IDs", placeholder = "[(trsansac_1, statemnt_1),(trsansac_2, statemnt_2)]", layout=widgets.Layout(width='600px'))
transaction_statement_list.style.description_width = '240px'
transaction_statement_list_combo = widgets.HBox([transaction_statement_list, create_tooltip("Enter the list of tuples of transaction and statements ids for the queries that you want to profile.")])
button_4 = widgets.Button(description="Get Query Plan")
output_4 = widgets.Output()
transaction_statement_list_val = None
target_schema_val_2 = None
key_val_2 = None



def is_input_valid(input_string):
    pattern = r'\[\((\d+),(\d+)\)(,\((\d+),(\d+)\))*\]'
    return bool(re.match(pattern, input_string))

def on_button_clicked_4(b):
    output_4.clear_output(wait=True)
    global target_schema_val_2, key_val_2, transaction_statement_list_val
    with output_4:
        if not is_input_valid(transaction_statement_list.value):
            print("\033[91m\033[1m Error \033[0m: Please check the input syntax. List of tuples is expected.")
        else:
            target_schema_val_2 = target_schema_2.value
            if not target_schema_val_2 == "" and not schema_exists(target_schema_val_2):
                print("\033[91m\033[1m Error! The schema does not exist. \033[0m\033[0m")
            else:
                key_val_2 = key_2.value
                transaction_statement_list_val = transaction_statement_list.value
                transaction_statement_list_val.replace(" ", "")
                logging.info(f"[Query Profile Main Page] For create using multiple transaction ids and keys, saved key value as :{key_val_2}, target schema as {target_schema_val_2}, List of transactions and statements as {transaction_statement_list_val}.")
                reset_values(transaction_statement_list_val = transaction_statement_list_val, target_schema_val_2 = target_schema_val_2, key_val_2 = key_val_2)

button_4.on_click(on_button_clicked_4)
create_list_transaction_statement = widgets.VBox([target_schema_2_combo, key_2_combo,transaction_statement_list_combo, button_4, output_4])

In [ ]:
tab_2 = widgets.Tab()
tab_2.children = [create_0, create_1, create_list_transaction_statement]
tab_2.titles = ["Transaction & Statement", "From SQL", "Multiple Queries"]

In [ ]:
# Vertical Border line
vertica_border_line = widgets.HTML("""
<style>
    .vertical-line {
        border-left: 2px solid lightgrey; /* Adjust thickness and color as needed */
        height: 400px; /* Adjust height as needed */
    }
</style>
</head>
<body>

<div class="vertical-line"></div>

</body>
""")

In [ ]:
right_heading = widgets.HTML("""<h1 style="text-align: left;">Create a new QueryProfiler</h1>""")
right_section = widgets.VBox([right_heading,search_box, tab_2],layout= {"margin" :"0px 0px 0px 50px"})
left_section

__________

In [ ]:
right_section

_____

In [ ]:
from IPython.display import HTML
logging.info(f"[Query Profile Main Page] Running script for HTML display")
display(
    HTML(
        """
<style>
.jupyter-widgets.widget-tab > .lm-TabBar .lm-TabBar-tab {
    flex: 0 1 auto
}
</style>
"""
    )
)

In [ ]:
# Utility function
def check_if_schema_key_exist(schema, key):
    query = f"""
        SELECT CASE WHEN COUNT(*) > 0 THEN 1 ELSE 0 END AS table_exists
        FROM v_catalog.tables 
        WHERE table_schema = '{schema}'
        AND table_name LIKE 'qprof_%{key}';
        """
    res = vp.vDataFrame(query)
    if res[0][0] == 1:
        return True
    else:
        return False

In [ ]:
%%html
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Version Number</title>
    <style>
        .version {
            font-family: Arial, sans-serif;
            font-size: 14px;
            color: #555;
            padding: 5px 10px;
            background-color: #f9f9f9;
            border: 1px solid #ddd;
            border-radius: 4px;
            display: inline-block;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="version">
        Page version update date: 8/23/2024
    </div>
</body>
</html>